In [1]:
import cobra
import csv

model = cobra.io.load_json_model('e_coli_core.json')

filename = 'e_coli_core_expression.csv'
estimated_maximal_activity = {}

with open(filename, mode='r', newline='') as file:
    reader = csv.reader(file)   
    for row in reader:
        
        try:
            num = float(row[1])
            estimated_maximal_activity[row[0]] = num
        except ValueError:
            continue


In [3]:
high_absolute_default_bound = 1000

for reaction in model.reactions:
    
    if (reaction.id=="ATPM"):
        print("ATPM reaction unchanged")
        continue
    
    if (reaction.id=="EX_glc__D_e"):
        print("EX_glc__D_e bounds changed to high absolute default bound")
        reaction.lower_bound = -high_absolute_default_bound
        reaction.upper_bound = high_absolute_default_bound
        continue
    
    try:
        value = estimated_maximal_activity[reaction.id]
    except KeyError:
        continue
    
    if (reaction.reversibility == True):
        reaction.lower_bound = -value
        reaction.upper_bound = value
        
    elif (reaction.reversibility == False):
        reaction.lower_bound = 0.0
        reaction.upper_bound = value

ATPM reaction unchanged
EX_glc__D_e bounds changed to high absolute default bound


In [4]:
for reaction in model.reactions:
    print(f'{reaction.id}: UB:{reaction.upper_bound}, LB:{reaction.lower_bound}')


PFK: UB:12.12, LB:0.0
PFL: UB:1.0, LB:0.0
PGI: UB:13.12, LB:-13.12
PGK: UB:23.13, LB:-23.13
PGL: UB:8.12, LB:0.0
ACALD: UB:1.16, LB:-1.16
AKGt2r: UB:3.1, LB:-3.1
PGM: UB:20.01, LB:-20.01
PIt2r: UB:6.03, LB:-6.03
ALCD2x: UB:9.01, LB:-9.01
ACALDt: UB:2.29, LB:-2.29
ACKr: UB:1.19, LB:-1.19
PPC: UB:2.56, LB:0.0
ACONTa: UB:25.35, LB:-25.35
ACONTb: UB:25.35, LB:-25.35
ATPM: UB:1000.0, LB:8.39
PPCK: UB:25.23, LB:0.0
ACt2r: UB:3.23, LB:-3.23
PPS: UB:2.5, LB:0.0
ADK1: UB:30.57, LB:-30.57
AKGDH: UB:24.35, LB:0.0
ATPS4r: UB:60.5, LB:-60.5
PTAr: UB:4.47, LB:-4.47
PYK: UB:26.78, LB:0.0
BIOMASS_Ecoli_core_w_GAM: UB:1000.0, LB:0.0
PYRt2: UB:1.26, LB:-1.26
CO2t: UB:30.45, LB:-30.45
RPE: UB:5.67, LB:-5.67
CS: UB:20.56, LB:0.0
RPI: UB:5.56, LB:-5.56
SUCCt2_2: UB:2.36, LB:0.0
CYTBD: UB:40.56, LB:0.0
D_LACt2: UB:4.56, LB:-4.56
ENO: UB:28.78, LB:-28.78
SUCCt3: UB:6.67, LB:0.0
ETOHt2r: UB:2.34, LB:-2.34
SUCDi: UB:24.34, LB:0.0
SUCOAS: UB:20.6, LB:-20.6
TALA: UB:4.45, LB:-4.45
THD2: UB:4.5, LB:0.0
TKT1: UB:3

In [8]:
import pandas as pd
from cobra.flux_analysis import flux_variability_analysis

pd.set_option('display.max_rows', None)
fva_results = flux_variability_analysis(model, model.reactions, fraction_of_optimum=1.0)

print("Flux Variability Results:")
print(fva_results)


Flux Variability Results:
                               minimum       maximum
PFK                       7.314456e+00  7.314456e+00
PFL                      -3.165524e-14  0.000000e+00
PGI                       7.376438e+00  7.376438e+00
PGK                      -1.446649e+01 -1.446649e+01
PGL                       4.751035e-01  4.751035e-01
ACALD                    -1.160000e+00 -1.160000e+00
AKGt2r                   -1.098154e-14  0.000000e+00
PGM                      -1.374906e+01 -1.374906e+01
PIt2r                     1.764199e+00  1.764199e+00
ALCD2x                   -1.160000e+00 -1.160000e+00
ACALDt                   -1.446793e-14 -4.784464e-15
ACKr                     -1.190000e+00 -1.190000e+00
PPC                       1.374260e+00  1.374260e+00
ACONTa                    3.289982e+00  3.289982e+00
ACONTb                    3.289982e+00  3.289982e+00
ATPM                      8.390000e+00  8.390000e+00
PPCK                      0.000000e+00  8.135426e-15
ACt2r               

In [ ]:

constrained_reactions = []
tolerance = 1e-9


for reaction in model.reactions:
    if reaction.id in estimated_maximal_activity:
        fva_max_flux = fva_results.loc[reaction.id, 'maximum']
        reaction_upper_bound = reaction.upper_bound
        
        if (reaction.id != 'FORt' and
            fva_max_flux > tolerance and  
            fva_max_flux < reaction_upper_bound - tolerance): 
            
            constrained_reactions.append(reaction.id)

print(f"Number of reactions limited by the network: {len(constrained_reactions)}")
print("\nList of constrained reaction IDs:")
print(constrained_reactions)

Number of reactions limited by the network: 30

List of constrained reaction IDs:
['PFK', 'PGI', 'PGL', 'PIt2r', 'PPC', 'ACONTa', 'ACONTb', 'AKGDH', 'ATPS4r', 'PTAr', 'PYK', 'CS', 'CYTBD', 'ENO', 'SUCDi', 'TALA', 'TKT1', 'TPI', 'FBA', 'FUM', 'G6PDH2r', 'GAPD', 'GLCpts', 'GLNS', 'GND', 'ICDHyr', 'MDH', 'NH4t', 'O2t', 'PDH']


Reaction behaviour explanation:
There are a total of 30 reactions that the permissible flux range in the forward direction is non-zero, but they come out less than the gene expression imposed upper flux bound. This is likely to be explained by the fact that a reactions maximum possible speed is limited by two things:
Their upper bond, so local capacity and the FVA result so in a sense their systemic throughput. That is the true maximum rate the reaction can occur given all other limitations in the network. We can say that these 30 reactions are not the bottleneck in the system but rather the network itself is the problem. Likely real bottlenecks would be upstream limits: for example the rate of glucose intake (GLCpts at 7,95) this limits any reaction downstream of that. Also the reactions themselves consuming its products are at their own maximum capacity. 

Based on the flux variablity results, there are a total of 40 reactions that have a positive minimum flux. For these reactions, it is essential that a reaction happens to fulfill the specified cellular objective, in this case it is maximum biomass production. They must carry a forward flux, thus cannot be turned off or reversed. For some of these reactions this is the case because they are the only link in the pathway that can produce a critical component in that pathway like a specific amino acid for example. Also the entire biomass reaction itself must be in a positive flux for the model to simulate and maintain growth. In some way, those specific reactions are essential to the cells metabolism. 